In [1]:
import pandas as pd
import numpy as np
from ast import literal_eval
import re
import glob, os
import requests
from bs4 import BeautifulSoup
from DataCleaning import get_nyt_review_text,get_omdb_awards

In [ ]:
# Scrape NYT movie reviews for full text:
# --- step completed; no need to re-run; commenting out for record-keeping

# nyt_files = glob.glob(os.path.join(r'data', "NYTData20*.csv"))
# nyt_df_temp = (pd.read_csv(f) for f in nyt_files)
# nyt_df   = pd.concat(nyt_df_temp, ignore_index=True)
# nyt_df = nyt_df.loc[:, ~nyt_df.columns.str.contains('^Unnamed')]
# nyt_df['review_text']=nyt_df['link'].apply(lambda x: get_nyt_review_text(literal_eval(x)['url']))
# nyt_df.to_csv("data/NYTData_wReviewText.csv",encoding='utf-8-sig')

# Read Cleaned CSV:
nyt_df=pd.read_csv("data/NYTData_wReviewText.csv", parse_dates=["publication_date","opening_date","date_updated"])
nyt_df = nyt_df.loc[:, ~nyt_df.columns.str.contains('^Unnamed')]
nyt_df=nyt_df[nyt_df["publication_date"].dt.year >= 2020] #Only 2020 and 2021 data

In [6]:
#Read OMDB Data from API calls:
omdb_files = glob.glob(os.path.join(r'data', "OMDBData202*.csv")) #Only 2020 and 2021 data
omdb_df_temp = (pd.read_csv(f,parse_dates=["Released","DVD"],thousands=',') for f in omdb_files)
omdb_df   = pd.concat(omdb_df_temp, ignore_index=True)
omdb_df = omdb_df.loc[:, ~omdb_df.columns.str.contains('^Unnamed')]

#Remove Duplicates based on unique imdbID:
omdb_df.drop_duplicates(["imdbID"],inplace=True)

#For movies with similar names, select more popular movie (more likely to be reviewed by NYT):
omdb_df=omdb_df.sort_values(by="imdbVotes",ascending=False,ignore_index=True)
omdb_df.drop_duplicates(["Title"],inplace=True)

# Clean OMDB Ratings:
omdb_df["Ratings"] = omdb_df["Ratings"].apply(literal_eval).apply(lambda x: dict([list(d.values()) for d in x]))
unpacked_ratings_df = omdb_df["Ratings"].apply(pd.Series)
omdb_df = omdb_df.join(unpacked_ratings_df)
omdb_df["Internet Movie Database"] = omdb_df["Internet Movie Database"].str.extract(r"(.*?)\/").astype(float)
omdb_df["Rotten Tomatoes"] = omdb_df["Rotten Tomatoes"].str.extract(r"(.*?)\%").astype(float)
omdb_df["Metacritic"] = omdb_df["Metacritic"].str.extract(r"(.*?)\/").astype(float)
omdb_df["Released"].dt.strftime('%Y-%m-%d');

#Convert runtimes to integer values
omdb_df["Runtime"]=omdb_df["Runtime"].apply(lambda x: re.findall(r"\d+",x)[0] if "min" in str(x) else x).astype(float)
omdb_df=omdb_df.rename({'Runtime': 'Runtime_mins'}, axis=1) #for clarity

#Get awards
omdb_df[['oscar_wins', 'oscar_noms','emmy_wins','emmy_noms','total_wins','total_noms']] = omdb_df.apply(get_omdb_awards, axis=1, result_type="expand")

#Convert Box Office numbers to float:
omdb_df["BoxOffice"] = omdb_df["BoxOffice"].replace('[\$,]', '', regex=True).astype(float)

In [ ]:
# Join OMDB and NYT DFs
joined_df=pd.merge(omdb_df, nyt_df,  how='inner', left_on=['Title'], right_on = ['display_title'])

#Write joined df:
joined_df.to_csv("data/joined_df.csv",encoding='utf-8-sig')